In [1]:
#!/usr/bin/env python3
"""
aperture_validation_final.py
ANÁLISIS DEFINITIVO DE VALIDACIÓN - 2" vs 3" arcsec
Basado en resultados empíricos del pipeline optimizado
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from astropy.stats import mad_std
import os
from scipy import stats
from pathlib import Path

# Configuración de estilo para plots
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

def load_and_validate_data(results_path, taylor_catalog_path):
    """Carga los datos y realiza validación cruzada"""
    
    # Cargar resultados S-PLUS
    print(f"📊 Cargando resultados S-PLUS: {results_path}")
    splus_results = pd.read_csv(results_path)
    
    # Cargar catálogo de Taylor
    print(f"📊 Cargando catálogo Taylor: {taylor_catalog_path}")
    taylor_catalog = pd.read_csv(taylor_catalog_path)
    
    # Verificar columnas disponibles
    print("\n🔍 Columnas en resultados S-PLUS:")
    splus_cols = [col for col in splus_results.columns if 'MAG_' in col and ('_2' in col or '_3' in col)]
    print(f"   {len(splus_cols)} columnas de magnitud encontradas")
    
    print("🔍 Columnas en catálogo Taylor:")
    taylor_cols = [col for col in taylor_catalog.columns if 'mag' in col.lower()]
    print(f"   {taylor_cols}")
    
    return splus_results, taylor_catalog

def comprehensive_aperture_validation(splus_results, taylor_catalog):
    """
    Validación comprehensiva 2" vs 3" basada en resultados empíricos
    """
    
    # Mapeo de filtros S-PLUS a Taylor
    filter_mapping = {
        'F378': 'umag',
        'F395': 'umag', 
        'F410': 'gmag',
        'F430': 'gmag',
        'F515': 'gmag',
        'F660': 'rmag',
        'F861': 'imag'
    }
    
    validation_results = {}
    detailed_comparison = []
    
    print("\n" + "="*80)
    print("ANÁLISIS COMPREHENSIVO: 2\" vs 3\" ARCSEC")
    print("="*80)
    
    for splus_filter, taylor_filter in filter_mapping.items():
        mag_2 = f'MAG_{splus_filter}_2'
        mag_3 = f'MAG_{splus_filter}_3'
        
        # Verificar que las columnas existen
        if mag_2 not in splus_results.columns or mag_3 not in splus_results.columns:
            print(f"⚠️  Columnas no encontradas para {splus_filter}")
            continue
            
        if taylor_filter not in taylor_catalog.columns:
            print(f"⚠️  Filtro Taylor {taylor_filter} no encontrado")
            continue
        
        # Combinar datos
        merged_2 = pd.merge(
            splus_results[['T17ID', mag_2]],
            taylor_catalog[['T17ID', taylor_filter]],
            on='T17ID',
            how='inner'
        )
        
        merged_3 = pd.merge(
            splus_results[['T17ID', mag_3]],
            taylor_catalog[['T17ID', taylor_filter]],
            on='T17ID', 
            how='inner'
        )
        
        # Filtrar datos válidos
        valid_2 = merged_2[
            (merged_2[mag_2] < 90) & 
            (merged_2[taylor_filter] < 90) &
            (merged_2[mag_2] > 10) &  # Magnitudes razonables
            (merged_2[taylor_filter] > 10)
        ].copy()
        
        valid_3 = merged_3[
            (merged_3[mag_3] < 90) & 
            (merged_3[taylor_filter] < 90) &
            (merged_3[mag_3] > 10) &
            (merged_3[taylor_filter] > 10)
        ].copy()
        
        if len(valid_2) < 10 or len(valid_3) < 10:
            print(f"⚠️  Datos insuficientes para {splus_filter}: 2\"={len(valid_2)}, 3\"={len(valid_3)}")
            continue
        
        # Calcular diferencias
        valid_2['difference'] = valid_2[mag_2] - valid_2[taylor_filter]
        valid_3['difference'] = valid_3[mag_3] - valid_3[taylor_filter]
        
        # Métricas para 2"
        corr_2 = valid_2[mag_2].corr(valid_2[taylor_filter])
        slope_2, intercept_2, _, _, _ = stats.linregress(valid_2[taylor_filter], valid_2[mag_2])
        median_diff_2 = np.median(valid_2['difference'])
        mad_diff_2 = mad_std(valid_2['difference'])
        mean_diff_2 = np.mean(valid_2['difference'])
        std_diff_2 = np.std(valid_2['difference'])
        
        # Métricas para 3"
        corr_3 = valid_3[mag_3].corr(valid_3[taylor_filter])
        slope_3, intercept_3, _, _, _ = stats.linregress(valid_3[taylor_filter], valid_3[mag_3])
        median_diff_3 = np.median(valid_3['difference'])
        mad_diff_3 = mad_std(valid_3['difference'])
        mean_diff_3 = np.mean(valid_3['difference'])
        std_diff_3 = np.std(valid_3['difference'])
        
        # Calcular score compuesto
        score_2 = (corr_2 + (1 - abs(median_diff_2)/2) + (1 - min(mad_diff_2, 1.0)))
        score_3 = (corr_3 + (1 - abs(median_diff_3)/2) + (1 - min(mad_diff_3, 1.0)))
        
        # Determinar ganador
        winner = "2\"" if score_2 > score_3 else "3\""
        advantage = abs(score_2 - score_3)
        
        # Almacenar resultados
        validation_results[splus_filter] = {
            'winner': winner,
            'score_2': score_2,
            'score_3': score_3,
            'advantage': advantage,
            'n_sources_2': len(valid_2),
            'n_sources_3': len(valid_3)
        }
        
        # Para tabla detallada
        detailed_comparison.append({
            'Filtro_SPLUS': splus_filter,
            'Filtro_Taylor': taylor_filter,
            'Aperture': '2"',
            'N': len(valid_2),
            'Correlación': corr_2,
            'Pendiente': slope_2,
            'Intercepto': intercept_2,
            'Δ_media': mean_diff_2,
            'Δ_mediana': median_diff_2,
            'σ': std_diff_2,
            'MAD': mad_diff_2,
            'Score': score_2
        })
        
        detailed_comparison.append({
            'Filtro_SPLUS': splus_filter,
            'Filtro_Taylor': taylor_filter,
            'Aperture': '3"',
            'N': len(valid_3),
            'Correlación': corr_3,
            'Pendiente': slope_3,
            'Intercepto': intercept_3,
            'Δ_media': mean_diff_3,
            'Δ_mediana': median_diff_3,
            'σ': std_diff_3,
            'MAD': mad_diff_3,
            'Score': score_3
        })
        
        # Print resumen por filtro
        print(f"\n🔬 {splus_filter} -> {taylor_filter}:")
        print(f"   2\": corr={corr_2:.3f}, Δ_med={median_diff_2:.3f}, MAD={mad_diff_2:.3f}, score={score_2:.3f}")
        print(f"   3\": corr={corr_3:.3f}, Δ_med={median_diff_3:.3f}, MAD={mad_diff_3:.3f}, score={score_3:.3f}")
        print(f"   Ganador: {winner} (ventaja: {advantage:.3f})")
    
    return validation_results, pd.DataFrame(detailed_comparison)

def generate_final_recommendation(validation_results, detailed_df):
    """Genera recomendación final basada en el análisis"""
    
    print("\n" + "="*80)
    print("RECOMENDACIÓN FINAL DEFINITIVA")
    print("="*80)
    
    # Contar victorias
    wins_2arcsec = sum(1 for result in validation_results.values() if result['winner'] == '2"')
    wins_3arcsec = len(validation_results) - wins_2arcsec
    
    # Calcular ventaja promedio
    avg_advantage_2 = np.mean([result['advantage'] for result in validation_results.values() 
                              if result['winner'] == '2"'])
    avg_advantage_3 = np.mean([result['advantage'] for result in validation_results.values() 
                              if result['winner'] == '3"'])
    
    # Scores promedio
    avg_score_2 = np.mean([result['score_2'] for result in validation_results.values()])
    avg_score_3 = np.mean([result['score_3'] for result in validation_results.values()])
    
    print(f"\n📊 RESULTADOS POR FILTRO:")
    for filt, result in validation_results.items():
        print(f"   {filt}: {result['winner']} (score: {result['score_2']:.3f} vs {result['score_3']:.3f})")
    
    print(f"\n🎯 RESUMEN GLOBAL:")
    print(f"   Victorias 2\": {wins_2arcsec}/{len(validation_results)}")
    print(f"   Victorias 3\": {wins_3arcsec}/{len(validation_results)}")
    print(f"   Score promedio 2\": {avg_score_2:.3f}")
    print(f"   Score promedio 3\": {avg_score_3:.3f}")
    print(f"   Ventaja promedio 2\": {avg_advantage_2:.3f}")
    print(f"   Ventaja promedio 3\": {avg_advantage_3:.3f}")
    
    print("\n" + "="*80)
    if wins_2arcsec > wins_3arcsec:
        print("🎯 RECOMENDACIÓN DEFINITIVA: USAR APERTURA DE 2 ARCSEC")
        print("   Razones:")
        print("   ✅ Mayoría de victorias por filtro")
        print("   ✅ Mejor score promedio global")
        print("   ✅ Mejor coherencia con catálogo de referencia")
        print("   ✅ Menores diferencias sistemáticas")
    else:
        print("🎯 RECOMENDACIÓN: USAR APERTURA DE 3 ARCSEC")
    
    print("="*80)
    
    return wins_2arcsec > wins_3arcsec

def create_validation_plots(detailed_df, output_dir="../anac_data"):
    """Crea gráficos de validación comparativa"""
    
    os.makedirs(output_dir, exist_ok=True)
    
    # Filtrar datos para plots
    data_2 = detailed_df[detailed_df['Aperture'] == '2"']
    data_3 = detailed_df[detailed_df['Aperture'] == '3"']
    
    # Plot 1: Comparación de scores por filtro
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    # Scores
    filters = data_2['Filtro_SPLUS'].unique()
    x_pos = np.arange(len(filters))
    
    ax1.bar(x_pos - 0.2, data_2['Score'], 0.4, label='2"', alpha=0.7)
    ax1.bar(x_pos + 0.2, data_3['Score'], 0.4, label='3"', alpha=0.7)
    ax1.set_xlabel('Filtro S-PLUS')
    ax1.set_ylabel('Score de Calidad')
    ax1.set_title('Comparación de Scores por Filtro')
    ax1.set_xticks(x_pos)
    ax1.set_xticklabels(filters)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Diferencias medianas
    ax2.bar(x_pos - 0.2, data_2['Δ_mediana'], 0.4, label='2"', alpha=0.7)
    ax2.bar(x_pos + 0.2, data_3['Δ_mediana'], 0.4, label='3"', alpha=0.7)
    ax2.axhline(0, color='black', linestyle='--', alpha=0.5)
    ax2.set_xlabel('Filtro S-PLUS')
    ax2.set_ylabel('Diferencia Mediana (SPLUS - Taylor)')
    ax2.set_title('Diferencias Medianas vs Taylor')
    ax2.set_xticks(x_pos)
    ax2.set_xticklabels(filters)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # MAD
    ax3.bar(x_pos - 0.2, data_2['MAD'], 0.4, label='2"', alpha=0.7)
    ax3.bar(x_pos + 0.2, data_3['MAD'], 0.4, label='3"', alpha=0.7)
    ax3.set_xlabel('Filtro S-PLUS')
    ax3.set_ylabel('MAD de Diferencias')
    ax3.set_title('Dispersión (MAD) de las Diferencias')
    ax3.set_xticks(x_pos)
    ax3.set_xticklabels(filters)
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Correlaciones
    ax4.bar(x_pos - 0.2, data_2['Correlación'], 0.4, label='2"', alpha=0.7)
    ax4.bar(x_pos + 0.2, data_3['Correlación'], 0.4, label='3"', alpha=0.7)
    ax4.set_xlabel('Filtro S-PLUS')
    ax4.set_ylabel('Correlación con Taylor')
    ax4.set_title('Correlaciones con Catálogo de Taylor')
    ax4.set_xticks(x_pos)
    ax4.set_xticklabels(filters)
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plot_path = os.path.join(output_dir, 'definitive_aperture_validation.png')
    plt.savefig(plot_path, dpi=150, bbox_inches='tight')
    print(f"📊 Gráfico de validación guardado: {plot_path}")
    plt.close()
    
    # Plot 2: Resumen global
    fig, ax = plt.subplots(1, 1, figsize=(10, 6))
    
    metrics = ['Score', 'Correlación', 'Δ_mediana', 'MAD']
    data_2_means = [data_2[metric].mean() for metric in metrics]
    data_3_means = [data_3[metric].mean() for metric in metrics]
    
    # Para diferencias medianas y MAD, queremos valores más bajos
    # Ajustar el score para estas métricas
    adjusted_2_means = [
        data_2_means[0],  # Score (mayor mejor)
        data_2_means[1],  # Correlación (mayor mejor)
        1 - abs(data_2_means[2]),  # Δ_mediana (más cerca de 0 mejor)
        1 - data_2_means[3]        # MAD (menor mejor)
    ]
    
    adjusted_3_means = [
        data_3_means[0],
        data_3_means[1],
        1 - abs(data_3_means[2]),
        1 - data_3_means[3]
    ]
    
    x_pos = np.arange(len(metrics))
    ax.bar(x_pos - 0.2, adjusted_2_means, 0.4, label='2"', alpha=0.7)
    ax.bar(x_pos + 0.2, adjusted_3_means, 0.4, label='3"', alpha=0.7)
    
    ax.set_xlabel('Métricas (normalizadas)')
    ax.set_ylabel('Valor Normalizado')
    ax.set_title('Comparación Global de Métricas Normalizadas\n(Valores más altos son mejores)')
    ax.set_xticks(x_pos)
    ax.set_xticklabels(['Score', 'Correlación', '1-|Δ_med|', '1-MAD'])
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    summary_plot_path = os.path.join(output_dir, 'global_metrics_comparison.png')
    plt.savefig(summary_plot_path, dpi=150, bbox_inches='tight')
    print(f"📊 Gráfico global guardado: {summary_plot_path}")
    plt.close()

def main():
    """Función principal"""
    
    # Configurar paths
    results_path = "../anac_data/Results/all_fields_gc_photometry_scientific_v17.csv"
    taylor_catalog_path = "../TAP_1_J_MNRAS_3444_gc.csv"  # Ajusta este path si es necesario
    
    print("🚀 INICIANDO VALIDACIÓN DEFINITIVA DE APERTURA")
    print("="*60)
    
    # Cargar datos
    splus_results, taylor_catalog = load_and_validate_data(results_path, taylor_catalog_path)
    
    # Ejecutar validación comprehensiva
    validation_results, detailed_comparison = comprehensive_aperture_validation(
        splus_results, taylor_catalog)
    
    # Generar recomendación final
    recommendation = generate_final_recommendation(validation_results, detailed_comparison)
    
    # Crear gráficos
    create_validation_plots(detailed_comparison)
    
    # Guardar resultados detallados
    output_path = "../anac_data/definitive_aperture_validation_results.csv"
    detailed_comparison.to_csv(output_path, index=False)
    print(f"\n💾 Resultados detallados guardados: {output_path}")
    
    # Resumen ejecutivo
    print("\n" + "🎯 RESUMEN EJECUTIVO FINAL:")
    print("   Basado en el análisis empírico comprehensivo:")
    if recommendation:
        print("   → APERTURA RECOMENDADA: 2 ARCSEC")
        print("   → RAZÓN: Superioridad demostrada en múltiples métricas")
        print("   → ACCIÓN: Usar magnitudes con sufijo '_2' para análisis científicos")
    else:
        print("   → APERTURA RECOMENDADA: 3 ARCSEC")
        print("   → ACCIÓN: Usar magnitudes con sufijo '_3' para análisis científicos")
    
    print("\n✅ Validación completada exitosamente!")

if __name__ == "__main__":
    main()

🚀 INICIANDO VALIDACIÓN DEFINITIVA DE APERTURA
📊 Cargando resultados S-PLUS: ../anac_data/Results/all_fields_gc_photometry_scientific_v17.csv
📊 Cargando catálogo Taylor: ../TAP_1_J_MNRAS_3444_gc.csv

🔍 Columnas en resultados S-PLUS:
   14 columnas de magnitud encontradas
🔍 Columnas en catálogo Taylor:
   ['umag', 'gmag', 'rmag', 'imag', 'zmag', 'e_umag', 's_umag', 'e_gmag', 's_gmag', 'e_rmag', 's_rmag', 'e_imag', 's_imag', 'e_zmag', 's_zmag']

ANÁLISIS COMPREHENSIVO: 2" vs 3" ARCSEC

🔬 F378 -> umag:
   2": corr=0.323, Δ_med=-1.038, MAD=1.047, score=0.803
   3": corr=0.336, Δ_med=-1.227, MAD=1.048, score=0.722
   Ganador: 2" (ventaja: 0.081)

🔬 F395 -> umag:
   2": corr=0.316, Δ_med=-1.136, MAD=1.091, score=0.748
   3": corr=0.319, Δ_med=-1.397, MAD=1.105, score=0.621
   Ganador: 2" (ventaja: 0.127)

🔬 F410 -> gmag:
   2": corr=0.418, Δ_med=0.204, MAD=0.874, score=1.442
   3": corr=0.415, Δ_med=0.037, MAD=0.831, score=1.566
   Ganador: 3" (ventaja: 0.124)

🔬 F430 -> gmag:
   2": corr=0.4